#ALL Functions

##import library

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import sys
sys.path.insert(0, '/content/drive/MyDrive/Team_Formation')

In [ ]:
import random
import numpy as np
from numpy import *

##Data Loading Function

##load skills connection

In [ ]:
#Function for Load Skills text file into system memory
def load_skills_in_memory( skills_file) :
  with open (skills_file, "r") as myfile:
    data = myfile.read().splitlines()
  return data


In [ ]:
#Process Skills Data and create skills matrix (verifier)
def process_skills_connection(skills_Data):
  persons_list=[];
  skills_list=[]
  unprocess_skills_connections=[];
  for idx in skills_Data:
    content = idx;
    process_already=False;
    results= content.split("=");
    for result in results:
      if result.strip() == "Total Persons".strip():
        total_person = int(results[1]);
        process_already=True;
      elif result.strip() == "Total Skills".strip():
        total_skills =int (results[1]);
        process_already=True;
      elif result.strip() == "Persons Mapping".strip():
        persons= results[1].split(",");
        for person in persons:
          ## need to create a string array
          persons_list.append(person);
        process_already=True;
      elif result.strip() == "Skills Mapping".strip() :
        skills= results[1].split(",");
        for skill in skills:
          ## need to create a string array
          skills_list.append(skill);
        process_already=True;
      else:
        if  content not in unprocess_skills_connections and process_already == False :
          unprocess_skills_connections.append(content);
  #Process the skills connection
  skills_connections = [[0] * (total_skills) for _ in range(total_person)]
  # initialize initial connections
  i = 0
  while (i < len(unprocess_skills_connections)) :
    s = unprocess_skills_connections[i].strip()
    result = s.split("=")
    val = result[1].split(":")
    j = 0
    while (j < len(val)) :
      skills_connections[i][j] = int(int(val[j]))
      j += 1
    i += 1
  return [total_person, total_skills, persons_list,skills_list, skills_connections];


##process cost connection

In [ ]:
#Function for Load cost text file into system memory
def load_cost_in_memory( cost_file) :
  with open (cost_file, "r") as myfile:
    data = myfile.read().splitlines()
  return data

In [ ]:
#Process Connection cost Data and create connection cost matrix(verifier)
def process_Connection_cost(cost_data,total_persons):
  unprocess_costs_connections=[];
  for data in cost_data:
    process_already= False;
    results= data.split('=');
    for result in results:
      str_value= result;
      if str_value.strip() == "Persons Mapping".strip():
        process_already = True;
        continue;
      else:
        if  data not in unprocess_costs_connections and process_already == False :
          unprocess_costs_connections.append(data);
  #process the connection cost
  connections_cost = [[0.0] * (total_persons) for _ in range(total_persons)]
  i = 0
  while (i < len(unprocess_costs_connections)) :
    s = unprocess_costs_connections[i].strip()
    result = s.split("=")
    val = result[1].split(":")
    j = 0
    while (j < len(val)) :
      connections_cost[i][j] = float(val[j]);
      j += 1
    i += 1
  return connections_cost;

##load skills to find

In [ ]:
#Function for Load required skills text file into system memory
def load_skills_to_find( req_file, skills_list) :
  match_index=[];
  with open (req_file, "r") as myfile:
    data = myfile.read().splitlines()
  text=data[0].split("=");
  skills= text[1].split(",")
  for skill in skills:
    if skill.strip() in skills_list: #if skills are in skills list
      index=skills_list.index(skill.strip());
      match_index.append(index);
    else: #if skill are not found
      print('Skills not found. Please check your Requiremnt text files');
      print("Skills not found =" , skill)
      sys.exit();
  match_index.sort();
  total_skills= len(skills_list);
  skills_to_find = [0] * (total_skills);
  i = 0
  while (i < total_skills) :
    if (i  in match_index) :
      # if match put 1 into that space
      skills_to_find[i] = 1
    i += 1

  return skills_to_find

##Objective Function

In [ ]:
#convert array sequence into a string
def array_sequence_to_string(array):
  sequence = ""
  i = 0
  while (i < len(array)) :
    if (i < len(array) - 1) :
      # for last item
      sequence = sequence + str(array[i]) + "-"
    else :
      # for all item
      sequence = sequence + str(array[i])
    i += 1
  return sequence;

In [ ]:
def  contain_skills_to_find( arr,  clone_skills_to_find) :
  outcome = False
  #print(arr)
  #print(clone_skills_to_find)
  x = 0
  while (x < len(clone_skills_to_find)) :
    if (clone_skills_to_find[x] == 1 and arr[x] == 1) :
      outcome = True
      #print('find some skill')
      clone_skills_to_find[x] = 0
    x += 1
  #print("outcome")
  #print(outcome)
  return outcome

In [ ]:
def  merge_elements_( arr1,  arr2) :
  merge_array = [0] * (total_skills)
  x = 0
  while (x < len(arr1)) :
    if (arr1[x] == arr2[x]) :
        merge_array[x] = arr1[x]
    elif(arr1[x] == 0 and arr2[x] == 1) :
        merge_array[x] = 1
    elif(arr1[x] == 1 and arr2[x] == 0) :
        merge_array[x] = 1
    x += 1
  return merge_array

In [ ]:
def  complete_merge_sequence_( arr,  skills_to_find) :
  outcome = True
  x = 0
  while (x < len(skills_to_find)) :
    if (skills_to_find[x] == 1 and arr[x] == 0) :
      outcome = False
      # if there are some/one skill needed
      break
    x += 1
  return outcome

In [ ]:
def objective_function(arr ,skills_to_find ):
  total_skills=len(skills_to_find);
  consist_skills_value = False
  # fill the int array by zero.
  trim_seq =  []
  clone_skills_to_find = skills_to_find ;
  for i in range(len(arr)):
    #print(i)
    consist_skills_value = False
    if (i == 0) :
      # for first iteration
      #print('i==0')
      person= arr[i];
      #print(person)
      update_answer = skills_connections[person]
      #print("persons skills")
      #print(update_answer)
      consist_skills_value = contain_skills_to_find(update_answer,clone_skills_to_find);
      #print("consist value" )
      #print( consist_skills_value )
      # check the person has that specific skills set
      if (consist_skills_value== True) :
        trim_seq.append(arr[i])
    else :
      #print('i>0')
      #print("person")
      person= arr[i];
      #print(person)
      # for all iteration
      tmp_answer=skills_connections[arr[i]]
      consist_skills_value = contain_skills_to_find(tmp_answer, clone_skills_to_find)
      if (consist_skills_value==True) :
        trim_seq.append(arr[i])
        update_answer = merge_elements_(update_answer, tmp_answer)
        if (complete_merge_sequence_(update_answer, skills_to_find)) :
          break;
    trim_result = trim_seq
  return trim_result;

In [ ]:
def objective_value1_( seq) :
  return (len(seq))

In [ ]:
def objective_value2_(seq,connections_cost) :
  cost = 0.0
  i = 0
  while (i < len(seq)) :
    row = seq[i]
    j = i + 1
    while (j < len(seq)) :
      col = seq[j]
      cost = (double)(cost + connections_cost[row][col]);
      j += 1
    i += 1
  #print("%.2d", cost)
  return (cost)

Testing objective function

##Transfer learning Functions

In [ ]:
def create_rank_fitness( fitness) :
  population_size= len(fitness);
  delta = 1.0E-4
  clone_fitness = fitness
  rank = [0] * (population_size)
  clone_fitness.sort()
  idx = 0
  i = 0
  while (i < len(fitness)) :
    j = 0
    while (j < len(fitness)) :
      if (abs(fitness[i] - clone_fitness[j]) < delta) :
        rank[idx] = j + 1
        clone_fitness[j] = sys.maxsize
        idx += 1
        break
      j += 1
    i += 1

  return rank

In [ ]:
def  Single_Scala_fitness( rank1,  rank2,  rank3,  rank4,  index) :
  # checked all ok no problem
  # compute new scalar fitness
  SF = 0
  # creating list.
  rankValue =  []
  rankValue.append(rank1[index])
  rankValue.append(rank2[index])
  rankValue.append(rank3[index])
  rankValue.append(rank4[index])
  minv = min(rankValue)
  SF = float(1.0) / minv
  rankValue.clear()
  return SF

In [ ]:
def  create_scala_fitness_matrix( rank1,  rank2,  rank3,  rank4) :
  # checked all ok no problem
  population_size = len(rank1);
  SF = [0.0] * population_size
  # creating list.
  rankValue =  []
  i = 0
  while (i < population_size) :
    rankValue.append(rank1[i])
    rankValue.append(rank2[i])
    rankValue.append(rank3[i])
    rankValue.append(rank4[i])
    minv = min(rankValue)
    SF[i] = float(1.0) / (minv)
    rankValue.clear()
    i += 1
  return SF

##Population initialization

In [ ]:
#genearate Random sequence
def generate_random_sequence(dimention):
  ar = [0] * (dimention)
  d = 0
  tmp = 0
  generator =  random;
  dummy_solution_array = [0] * (dimention)
  counter = 0
  while (counter < dimention) :
      dummy_solution_array[counter] = counter
      counter += 1
  # copy array from seq_arr
  ar = dummy_solution_array;
  # swap with new ar with random index
  # the first index and last index must not change
  i = 0
  while (i < dimention - 1) :
    d = i + (generator.randint(0, (dimention - 1 - i)));
    # generate random number between i and (dim-1-i)
    # System.out.print("generator "+ d);
    tmp = ar[i]
    ar[i] = ar[d]
    ar[d] = tmp
    i += 1
  return ar

In [ ]:
def initialize_population(task) :
  population = [[0] * (total_person) for _ in range(population_size)]
  # Task-1
  obj_value1_array = [0] * (population_size)
  obj_value2_array = [0.00] * (population_size)
  population_short_seq_list = [None] * (population_size)

  # Generate Random Population and copy it intp population variable
  arr = None
  row = 0
  while (row < population_size) :
    # generate long sequence
    arr = generate_random_sequence(total_person)
    # generate random sequence with random numbers between 1 to total no of person
    population[row] = arr
    # evaluate the population by calculating fitness
    task_1_fitness = objective_function(arr, task.copy())
    population_short_seq_list[row] = array_sequence_to_string(task_1_fitness)
    task1_obj_person = objective_value1_(task_1_fitness)
    obj_value1_array[row] = task1_obj_person
    task1_obj_costs = objective_value2_(task_1_fitness,connections_cost)
    obj_value2_array[row] = (double)(task1_obj_costs)
    print('population ', row)
    print('population ', population[row])
    print("team sequence ", population_short_seq_list[row])
    print("team size ", task1_obj_person)
    print("cost" , task1_obj_costs)
    row+=1


#Main function

In [ ]:
#Main program controller
##Load Skills memeory into system
skills_file= "/content/drive/MyDrive/Team Formation/ExtractedSkill_Staff_Expertise_Data.txt";
skills_data=load_skills_in_memory(skills_file);
##process the skills memory and create skills matrix
total_person, total_skills, persons_list,skills_list,skills_connections=process_skills_connection(skills_data);
'''
print('Total Person = ', total_person)
print('Total Skills = ', total_skills)
print('Person list ' , persons_list)
print('Skills list ', skills_list)
print('Skills Connection ' , skills_connections)
'''
#initialize skills collection and load into system
cost_file =  '/content/drive/MyDrive/Team Formation/ConnectionCost_Staff_Expertise_Data.txt';
cost_data = load_cost_in_memory(cost_file);
#process the cost data and create cost matrix
connections_cost = process_Connection_cost(cost_data, total_person);
#print('connection cost matrix', connections_cost)

#load req skills
skills1=load_skills_to_find("/content/drive/MyDrive/Team Formation/Required skills 20_1.txt", skills_list);
#skills2=load_skills_to_find("/content/drive/MyDrive/Team_Formation/Required skills 20_2.txt", skills_list);
#skills3=load_skills_to_find("/content/drive/MyDrive/Team_Formation/Required skills 10_3.txt", skills_list);
#skills4=load_skills_to_find("/content/drive/MyDrive/Team_Formation/Required skills 20_4.txt", skills_list);

####initialize population
population_size=100;
initialize_population(skills1.copy())

#jaya Algorithm